<a href="https://colab.research.google.com/github/punkmic/pyspark-concepts/blob/master/Read_Write_and_Validate_Data_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading, Writing and Validating Data in PySpark HW

Welcome to your first coding homework assignment in PySpark! I hope you enjoyed the lecture on Reading, Writing and Validating dataframes. Now it's time to put what you've learned into action! 

I've included several instructions below to help guide you through this homework assignment which I hope will get you feeling even comfortable reading, writing and validating dataframes. If you get stuck at any point, feel free to jump to the next lecture where I will guide you through my solutions to the HW assignment. 

Have fun!

Let's dig right in!


## But first things first.....
We need to always begin every Spark session by creating a Spark instance. Let's go ahead and use the method we learned in the lecture in the cell below. Also see if you can remember how to open the Spark UI (using a link that automatically guides you there). 

In [3]:
try:
  import pyspark
  from pyspark.sql import SparkSession
except:
  !pip install pyspark
  import pyspark

In [4]:
spark = SparkSession.builder.appName('spark').getOrCreate()

In [6]:
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print(f'Cores: {cores}')

Cores: 1


In [9]:
spark

## Next let's start by reading a basic csv dataset

Download the pga_tour_historical dataset that is attached to this lecture and save it whatever folder you want, then read it in. 

**Data Source:** https://www.kaggle.com/bradklassen/pga-tour-20102018-data

Rememer to try letting Spark infer the header and infer the Schema types!

In [10]:
path = '/content/pga_tour_historical.csv'
pga = spark.read.csv(path=path, inferSchema=True, header=True)

## 1. View first 5 lines of dataframe
First generate a view of the first 5 lines of the dataframe to get an idea of what is inside. We went over two ways of doing this... see if you can remember BOTH ways. 

In [11]:
pga.show()

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
|      John Daly|  2010|Driving Distance|Driving Distance ...|   63|
|  Graham DeLaet|  2010|Driving Distance|Driving Distance ...|   88|
|  Angel Cabrera|  2010|Driving Distance|Driving Distance ...|   64|
| Charles Warren|  2010|Driving Distance|Driving Distance ...|   64|
|    D.J. Trahan|  2010|Driving Distance|Driving Distance ...|   92|
|  Martin Flores|  2010|Driving Distance|Driving Distance ...|   75|
|   Rory McIlroy|  2010|Driving Di

## 2. Print the schema details

Now print the details of the dataframes schema that Spark infered to ensure that it was infered correctly. Sometimes it is not infered correctly, so we need to watch out!

In [14]:
print(f'Schema: {pga.printSchema()}')
print(f'\nColumns: {pga.columns}')

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: string (nullable = true)

Schema: None

Columns: ['Player Name', 'Season', 'Statistic', 'Variable', 'Value']


## 3. Edit the schema during the read in

We can see from the output above that Spark did not correctly infer that the "value" column was an integer value. Let's try specifying the schema this time to let spark know what the schema should be.

Here is a link to see a list of PySpark data types in case you need it (also attached to the lecture): 
https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [15]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [17]:
data_schema = [StructField('Player Name', StringType(), True),
               StructField('Season', IntegerType(), True),
               StructField('Statistic', StringType(), True),
               StructField('Variable', StringType(), True),
               StructField('Value', IntegerType(), True)]

In [20]:
final_schema = StructType(fields=data_schema)

In [21]:
pga = spark.read.csv(path=path, schema=final_schema)

In [23]:
pga.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: integer (nullable = true)



## 4. Generate summary statistics for only one variable

See if you can generate summary statistics for only the "Value" column using the .describe function

(count, mean, stddev, min, max) 

In [25]:
pga.describe(['Value']).show()

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|           1471110|
|   mean|12353.530796473411|
| stddev|156400.06424154204|
|    min|              -178|
|    max|           3564954|
+-------+------------------+



## 5. Generate summary statistics for TWO variables
Now try to generate ONLY the count min and max for BOTH the "Value" and "Season" variable using the select. You can't use the .describe function for this one but see if you can remember which function you CAN use. 

In [26]:
pga.select('Value', 'Season').summary("count", "max", "min").show()

+-------+-------+-------+
|summary|  Value| Season|
+-------+-------+-------+
|  count|1471110|2435377|
|    max|3564954|   2018|
|    min|   -178|   2010|
+-------+-------+-------+



## 6. Write a parquet file

Now try writing a parquet file (not partitioned) from the pga dataset. But first create a new dataframe containing ONLY the the "Season" and "Value" fields (using the "select command you used in the question above) and write a parquet file partitioned by "Season". This is a bit of a challenge aimed at getting you ready for material that will be covered later on in the course. Don't feel bad if you can't figure it out.

*Note that if any of your variable names contain spaces, spark will produce an error message with this call. That is why we are selecting ONLY the "Season" and "Value" fields. Ideally we should renamed those columns but we haven't gotten to that yet in this course but we will soon!*

In [33]:
df = pga.select('Season', 'Value')
df.write.mode('overwrite').parquet('partition_parquet/')
df.show(3)

+------+-----+
|Season|Value|
+------+-----+
|  null| null|
|  2010|   71|
|  2010|   77|
+------+-----+
only showing top 3 rows



## 7. Write a partioned parquet file

You will need to use the same limited dataframe that you created in the previous question to accomplish this task as well. 

In [34]:
df.write.mode('overwrite').partitionBy('Season').parquet('partioned_parquet/')
df.show(3)

+------+-----+
|Season|Value|
+------+-----+
|  null| null|
|  2010|   71|
|  2010|   77|
+------+-----+
only showing top 3 rows



## 8. Read in a partitioned parquet file

Now try reading in the partitioned parquet file you just created above. 

In [35]:
path = '/content/partioned_parquet'
parquet = spark.read.parquet(path)
parquet.show(3)

+-----+------+
|Value|Season|
+-----+------+
|   71|  2010|
|   77|  2010|
|   83|  2010|
+-----+------+
only showing top 3 rows



## 9. Reading in a set of paritioned parquet files

Now try only reading Seasons 2010, 2011 and 2012.

In [37]:
path = '/content/partioned_parquet/'
partitioned = spark.read.parquet(path+'Season=2010/', path+'Season=2011/', path+'Season=2012/')
partitioned.show(3)

+-----+
|Value|
+-----+
|   71|
|   77|
|   83|
+-----+
only showing top 3 rows



## 10. Create your own dataframe

Try creating your own dataframe below using PySparks *.createDataFrame* function. See if you can make one that contains 4 variables and at least 3 rows. 

Let's see how creative you can get on the content of the dataframe :)

In [38]:
values = [('Chloe Price', 20), ('Max Caulfield', 19), ('Rachel  Amber', 20)]
df = spark.createDataFrame(values, ['Name', 'Age'])
df.show(3)

+-------------+---+
|         Name|Age|
+-------------+---+
|  Chloe Price| 20|
|Max Caulfield| 19|
|Rachel  Amber| 20|
+-------------+---+



## We're done! Great job!